In [1]:
from tkinter import *
from tkinter import ttk, filedialog, messagebox
import threading
import os
from reportlab.lib.pagesizes import letter
from reportlab.lib.styles import getSampleStyleSheet
from reportlab.platypus import SimpleDocTemplate, Paragraph
from reportlab.pdfbase import pdfmetrics
from reportlab.pdfbase.ttfonts import TTFont
import PyPDF2
import docx
from googletrans import Translator, LANGUAGES
from gtts import gTTS
import openpyxl
from pptx import Presentation
import speech_recognition as sr

def read_pdf(file_path):
    with open(file_path, 'rb') as file:
        pdf_reader = PyPDF2.PdfReader(file)
        text = ''
        for page_number in range(len(pdf_reader.pages)): 
            page = pdf_reader.pages[page_number]
            text += page.extract_text()
    return text

def read_docx(file_path):
    doc = docx.Document(file_path)
    text = ''
    for paragraph in doc.paragraphs:
        text += paragraph.text + '\n'
    return text

def read_excel(file_path):
    wb = openpyxl.load_workbook(file_path)
    sheet = wb.active
    text = ''
    for row in sheet.iter_rows(values_only=True):
        for cell in row:
            if cell:
                text += str(cell) + '\n'
    return text

def read_pptx(file_path):
    prs = Presentation(file_path)
    text = ''
    for slide in prs.slides:
        for shape in slide.shapes:
            if hasattr(shape, "text"):
                text += shape.text + '\n'
    return text

def upload_file():
    file_path = filedialog.askopenfilename(filetypes=[('PDF Files', '*.pdf'), 
                                                      ('DOC Files', '*.doc;*.docx'),
                                                      ('Excel Files', '*.xlsx'),
                                                      ('PowerPoint Files', '*.pptx')])
    if file_path:
        file_content = ''
        if file_path.lower().endswith('.pdf'):
            file_content = read_pdf(file_path)
        elif file_path.lower().endswith('.doc') or file_path.lower().endswith('.docx'):
            file_content = read_docx(file_path)
        elif file_path.lower().endswith('.xlsx'):
            file_content = read_excel(file_path)
        elif file_path.lower().endswith('.pptx'):
            file_content = read_pptx(file_path)
        
        Input_text.delete(1.0, END)
        Input_text.insert(END, file_content)

def translate_text():
    def _translate():
        translator = Translator()
        input_text = Input_text.get("1.0", "end-1c")

        dest_language = dest_lang.get()

        try:
            progress_bar.start()
            if dest_language in LANGUAGES.values():
                translated = translator.translate(text=input_text, dest=dest_language)
                translated_text = translated.text

                Output_text.delete(1.0, END)
                Output_text.insert(END, f"Translated Text ({dest_language}):\n{translated_text}")
                translate_label.config(text="Translation completed.", fg="green")
            else:
                Output_text.delete(1.0, END)
                Output_text.insert(END, "Invalid destination language code.")
                translate_label.config(text="Invalid language code.", fg="red")
        except Exception as e:
            Output_text.delete(1.0, END)
            Output_text.insert(END, f"Translation Error: {e}")
            translate_label.config(text="Translation error.", fg="red")
        finally:
            progress_bar.stop()

    threading.Thread(target=_translate).start()


def download_pdf():
    translated_text = Output_text.get(1.0, END)
    file_path = filedialog.asksaveasfilename(defaultextension=".pdf", filetypes=[("PDF Files", "*.pdf")])
    
    if file_path:
        try:
            lucida_font_path = r'C:\Windows\Fonts\l_10646.ttf'  # Path to Lucida Unicode Regular font
            pdfmetrics.registerFont(TTFont('LucidaUnicode', lucida_font_path))

            doc = SimpleDocTemplate(file_path, pagesize=letter)
            styles = getSampleStyleSheet()
            flowables = []

            translated_paragraphs = translated_text.split('\n')
            for para in translated_paragraphs:
                if para.strip():
                    para_obj = Paragraph(para.strip(), styles['Normal'])
                    flowables.append(para_obj)

            styles['Normal'].fontName = 'LucidaUnicode'
            styles['Normal'].fontSize = 10

            doc.build(flowables)
            messagebox.showinfo("Success", "PDF downloaded successfully!")
        except Exception as e:
            messagebox.showerror("Error", f"Error in downloading PDF: {e}")

def text_to_speech():
    input_text = Output_text.get(1.0, END)
    if not input_text.strip():
        messagebox.showwarning("Warning", "No text to convert to speech!")
        return
    
    language = 'en'
    speech = gTTS(text=input_text, lang=language, slow=False)
    speech.save("output.mp3")
    os.system("start output.mp3")

def speech_to_text_background():
    def _recognize_speech():
        recognizer = sr.Recognizer()
        with sr.Microphone() as source:
            recognizer.adjust_for_ambient_noise(source)
            speak_label.config(text="Speak now...", fg="green")
            audio = recognizer.listen(source)

        try:
            text = recognizer.recognize_google(audio)
            Input_text.delete(1.0, END)
            Input_text.insert(END, text)
            speak_label.config(text="")
            translate_text()
        except Exception as e:
            speak_label.config(text="")

    thread = threading.Thread(target=_recognize_speech)
    thread.start()

def show_welcome_message():
    messagebox.showinfo("Welcome", "Welcome to the Advanced Language Translator App!")


root = Tk()
root.geometry('800x600')
root.title('Advanced Language Translator')
root.configure(bg='lightblue')

style = ttk.Style()
style.configure('TLabel', font=('Arial', 12), background='lightblue')
style.configure('TButton', font=('Arial', 12), padding=10, background='#1976D2', foreground='black')  # Change button color here
style.map('TButton', background=[('active', '#1565C0')])
style.configure('TCombobox', font=('Arial', 12))
style.configure('TText', font=('Arial', 12))

main_frame = Frame(root, bg='lightblue')
main_frame.pack(pady=10, padx=10, fill=BOTH, expand=True)

top_frame = Frame(main_frame, bg='lightblue')
top_frame.pack(fill=X)
Label(top_frame, text="Advanced Language Translator", font=("Arial", 24, 'bold'), bg='lightblue',fg='black').pack(pady=10)

Label(top_frame, text="Enter Text or Upload File", font=("Arial", 14, 'bold'), bg="white").pack(anchor='w', padx=10, pady=5)
Input_text = Text(top_frame, font=("Arial", 12), height=8, wrap=WORD, padx=8, pady=8, width=70)
Input_text.pack(padx=10)

Label(top_frame, text="Translated Text", font=("Arial", 14, 'bold'), bg="black", fg="white").pack(anchor='w', padx=10, pady=(10, 5))
Output_text = Text(top_frame, font=("Arial", 12), height=8, wrap=WORD, padx=10, pady=10, width=70)
Output_text.pack(padx=10)

language = list(LANGUAGES.values())
dest_lang = ttk.Combobox(top_frame, values=language, width=30)
dest_lang.pack(padx=10, pady=(10, 10))
dest_lang.set("Choose Language")

btn_frame = Frame(main_frame, bg='lightblue')
btn_frame.pack(pady=10)

upload_btn = ttk.Button(btn_frame, text='Upload File', command=upload_file, style='TButton')
upload_btn.grid(row=0, column=0, padx=10, pady=10)

trans_btn = ttk.Button(btn_frame, text='Translate', command=translate_text, style='TButton')
trans_btn.grid(row=0, column=1, padx=10, pady=10)

download_pdf_btn = ttk.Button(btn_frame, text='Download PDF', command=download_pdf, style='TButton')
download_pdf_btn.grid(row=0, column=2, padx=10, pady=10)

text_to_speech_btn = ttk.Button(btn_frame, text='Text to Speech', command=text_to_speech, style='TButton')
text_to_speech_btn.grid(row=0, column=3, padx=10, pady=10)

speech_to_text_btn = ttk.Button(btn_frame, text='Speech to Text', command=speech_to_text_background, style='TButton')
speech_to_text_btn.grid(row=0, column=4, padx=10, pady=10)

progress_bar = ttk.Progressbar(btn_frame, orient=HORIZONTAL, length=300, mode='indeterminate')
progress_bar.grid(row=1, columnspan=5, pady=10)

speak_label = Label(main_frame, text="", font=("Arial", 12), fg="green", bg="lightblue")
speak_label.pack(pady=(10, 0))

translate_label = Label(btn_frame, text="", font=("Arial", 12), bg="lightblue")
translate_label.grid(row=2, columnspan=5, padx=10, pady=10)

def display_welcome_message():
    show_welcome_message()

root.after(1500, display_welcome_message)  

root.mainloop()

